
## Libraries

In [250]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split


## Preparing The Data


In [261]:
class MyData(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.data['University_Year'] = self.data['University_Year'].str[0].astype(int)
        self.features = self.data.drop(columns=['Student_ID', 'Gender', 'Sleep_Duration', 'Sleep_Quality',
                                                'Weekday_Sleep_Start', 'Weekday_Sleep_End', 'Weekend_Sleep_Start',
                                                'Weekend_Sleep_End'])
        self.labels = self.data['Sleep_Duration']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the sample and corresponding label
        sample = torch.tensor(self.features.iloc[idx].values, dtype=torch.float32)
        label = torch.tensor(self.labels.iloc[idx], dtype=torch.float32)
        return sample, label

Sleep = MyData('/content/student_sleep_patterns.csv')

# Divinding the data into training, validation and test sets
train_size = int(0.8 * len(Sleep))              # 80%
val_size = int(0.1 * len(Sleep))                # 10%
test_size = len(Sleep) - train_size - val_size  # 10%
train_data, val_data, test_data = random_split(Sleep, [train_size, val_size, test_size])

# Further dividing the sets into mini-batches
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


## Neural Network

In [265]:
class MyNN(nn.Module):
    def __init__(self):
        super(MyNN, self).__init__()
        self.fc1 = nn.Linear(6, 80)          # Input layer (6 features per sample)
        self.fc2 = nn.Linear(80, 50)         # Hidden layer 1 (80 neurons)
        self.fc3 = nn.Linear(50, 1)          # Hidden layer 2 (50 neurons) - Outputs a scalar

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = MyNN()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)
epochs = 150

for epoch in range(epochs):
    model.train()                         # Sets model to training mode
    running_loss = 0

    for features, labels in train_loader:
        optimizer.zero_grad()             # Resets gradient values
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    average_loss = running_loss / train_size
    #print(average_loss)


## Runnning the Model on Validation Set

In [266]:
def evaluation(loader):
    model.eval()                   # Sets model to evaluation mode

    with torch.no_grad():          # Disables gradient calculations
        for features, labels in loader:
          output = model(features)
          batch_percentage_diff = ((labels - output).abs() / labels) * 100
          avg_batch_percentage_diff = batch_percentage_diff.mean().item()
    return avg_batch_percentage_diff

percentage_differences = evaluation(val_loader)
print(f'Percentage differences: {percentage_differences}')

Percentage differences: 25.736026763916016



## Runnning the Model on Test Set

In [267]:
percentage_differences = evaluation(test_loader)
print(f'Percentage differences: {percentage_differences}')

Percentage differences: 20.480247497558594
